In [4]:
import os
import sys

s3_input_files_address = "s3://ucsd-ccbb-interns/Mustafa/chip_test"
s3_output_files_address = "s3://ucsd-ccbb-interns/Mustafa/chip_test/histone_no_pairs"

## CFNCluster name
your_cluster_name = "mustafa5"

## The private key pair for accessing cluster.
private_key = "/home/mustafa/interns_oregon_key.pem"

## Project information
project_name = "mus_test_proj"

#homer
workflow = "homer"

genome = "hg19"

style = "histone"

## If delete cfncluster after job is done.
delete_cfncluster= False

print("variables set")

variables set


In [5]:
sys.path.append("../../src/cirrus_ngs")
from cfnCluster import CFNClusterManager, ConnectionManager

## Create a new cluster
master_ip_address = CFNClusterManager.create_cfn_cluster(cluster_name=your_cluster_name)
ssh_client = ConnectionManager.connect_master(hostname=master_ip_address,
               username="ec2-user",
               private_key_file=private_key)

cluster mustafa5 does exist.

Status: CREATE_COMPLETE
Status: CREATE_COMPLETE
MasterServer: RUNNING
MasterServer: RUNNING
Output:"MasterPublicIP"="52.43.127.225"
Output:"MasterPrivateIP"="172.31.27.132"
Output:"GangliaPublicURL"="http://52.43.127.225/ganglia/"
Output:"GangliaPrivateURL"="http://172.31.27.132/ganglia/"

connecting
connected


In [6]:
from util import PipelineManager
from util import DesignFileLoader

##can be fastqc, notrim, bwa
analysis_steps = {
                    "fastqc"
                    ,"bowtie"
                    ,"make_tag_directory"
                    ,"make_UCSC_file"
                    ,"find_peaks"
                    ,"annotate_peaks"
                    ,"pos2bed"
                    ,"find_motifs_genome"
                }

design_file = "/home/mustafa/ccbb/cirrus-ngs/data/cirrus-ngs/testing_chip_no_pairs.txt"


sample_list, group_list, pairs_list = DesignFileLoader.load_design_file(design_file)

PipelineManager.execute("ChiPSeq", ssh_client, project_name, analysis_steps, s3_input_files_address,
                       sample_list, group_list, s3_output_files_address, genome, style, pairs_list, workflow)

[['chipseq_sample1_chip.fastq.gz'], ['chipseq_sample1_input.fastq.gz'], ['chipseq_sample2_chip.fastq.gz'], ['chipseq_sample2_input.fastq.gz']]
['groupA', 'groupA', 'groupB', 'groupB']
{}
making the yaml file...
copying yaml file to remote master node...
mus_test_proj.yaml
/shared/workspace/Pipelines/yaml_files/ChipSeq
executing pipeline...
Executing qsub -V -o /dev/null -e /dev/null /shared/workspace/Pipelines/scripts/run.sh /shared/workspace/Pipelines/yaml_files/ChipSeq/mus_test_proj.yaml /shared/workspace/logs/ChipSeq/mus_test_proj ChipSeq
